In [1]:
import pandas as pd
import numpy as np
import datetime

In [68]:
budgetExcel = pd.concat(
    pd.read_excel("TALOUSARVIO 23.xlsx", sheet_name=None, header=[ 10, 11]),
    ignore_index=True,
)
if len(budgetExcel.columns) != 31:
    raise Exception("Sheets don't follow the same pattern")
# Dropping columns we don't need
budgetExcel = budgetExcel.drop(
    budgetExcel.columns[[3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]],
    axis=1,
)
# replace yes and no in the housing column
budgetExcel.iloc[:, 2] = budgetExcel.iloc[:, 2].map({"K": True, "E": False})

# CHANGE THIS HERE TO ONLY GET PROJECTS AND NOT MISS ANY PROJECT
budgetExcel = budgetExcel[budgetExcel[budgetExcel.columns[15]].notna() | budgetExcel[budgetExcel.columns[1]].notna()| budgetExcel[budgetExcel.columns[2]].notna() | budgetExcel[budgetExcel.columns[14]].notna()]
budgetExcel = budgetExcel.replace({np.nan: None, "?": None})


# currYear = datetime.date.today().year
# financeShift = list(budget_23.columns.get_level_values(0))[4] - currYear - 1
# if financeShift != 0:
#     finance_df = budget_23.iloc[:, 4:14]
#     budget_23.iloc[:, 4:14] = finance_df.shift(financeShift, axis=1)

# projects with Ids available
budgetExcel_Ids=budgetExcel[budgetExcel[budgetExcel.columns[15]].apply(type)==int]
# projects with no Ids available
budgetExcel_no_Ids = budgetExcel[budgetExcel[budgetExcel.columns[15]].apply(type)!=int]



budgetExcel_no_Ids

Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                Unnamed: 0_level_1 Unnamed: 1_level_1   
18                    Hakaniemi rinnakkaishankkeet               K5.1   
26                  Merihaan täydennysrakentaminen               K5.1   
30          Savonkadun alue (Pasilan projektialue)               None   
31                                      Hangonkatu               K5.1   
34                  Itä-Pasilan Mäkelänkadun reuna               None   
..                                             ...                ...   
619                                     Mätäspolku                 K4   
620  Maatullinkj (Henrik Forsiuksen t - Kämnerink)                 K4   
624                      Muut peruskorjaushankkeet                 K4   
625                        Pienet peruskorjaustyöt                 K4   
626                   Katupuiden korvausistutukset                 K4   

    asuntotuotan. ennuste   2023   2024   2025   2026   2027   2028   2029  \
       Kyllä / Ei  1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 €   
18           None   400.0   None   None   None   None   None   None   None   
26           None  1000.0   None   None   None   None   None   None   None   
30           None    None   None   None   None   None   None   None   None   
31           None   200.0   None   None   None   None   None   None   None   
34           None  1000.0   None   None   None   None   None   None   None   
..            ...     ...    ...    ...    ...    ...    ...    ...    ...   
619         False   400.0   None   None   None   None   None   None   None   
620         False   900.0   None   None   None   None   None   None   None   
624         False  2000.0   None    0.0   None   None   None   None   None   
625         False  1000.0   None   None   None   None   None   None   None   
626         False   200.0   None   None   None   None   None   None   None   

      2030   2031   2032             HKR / PW  
    1000 € 1000 € 1000 € 1000 €.1 HANKENUMERO  
18    None   None   None     None        None  
26    None   None   None     None        None  
30    None   None   None     None        None  
31    None   None   None     None        None  
34    None   None   None     None        None  
..     ...    ...    ...      ...         ...  
619   None   None   None     None        None  
620   None   None   None     None        None  
624   None   None   None     None        None  
625   None   None   None     None        None  
626   None   None   None     None        None  

[225 rows x 16 columns]

In [69]:
planExcel = pd.concat(
    pd.read_excel("TOIMINTASUUNNITELMA 23.xlsx", sheet_name=None, header=[0, 1]),
    ignore_index=True,
)
if len(planExcel.columns) != 28:
    raise Exception("Sheets don't follow the same pattern")

planExcel = planExcel.drop(
    planExcel.columns[
        [5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
    ],
    axis=1,
)

# Replacing all common " with related network and project Ids
planExcel.iloc[:, 1] = planExcel.iloc[:, 1].dropna().replace(["\""], method="ffill").reindex(planExcel.iloc[:, 1].index)
planExcel.iloc[:, 2] = planExcel.iloc[:, 2].dropna().replace(["\""], method="ffill").reindex(planExcel.iloc[:, 2].index)

# Dropping all non project rows
planExcel = planExcel[planExcel[planExcel.columns[1]].notna() | planExcel[planExcel.columns[2]].notna() | planExcel[planExcel.columns[3]].notna()| planExcel[planExcel.columns[4]].notna() | planExcel[planExcel.columns[7]].notna()]
planExcel = planExcel.replace({np.nan: None, "?": None})

# plan_23.groupby(plan_23.columns[7]).aggregate({plan_23.columns[1]: lambda x: list(x)})




In [70]:
# merging budget projects with plan projects on same PW id
merged_Plan_Budget = pd.merge(
    budgetExcel_Ids,
    planExcel,
    left_on=[budgetExcel_Ids.columns[15]],
    right_on=[planExcel.columns[7]],
    how="inner",
)
# Fixing data discrepancy between similar columns
merged_Plan_Budget.iloc[:, 4] = merged_Plan_Budget.iloc[:, 21]
merged_Plan_Budget.iloc[:, 0] = merged_Plan_Budget.iloc[:, 16]
merged_Plan_Budget.iloc[:, 14] = merged_Plan_Budget.iloc[:, 22]
# Dropping duplicated columns
merged_Plan_Budget = merged_Plan_Budget.drop(
    merged_Plan_Budget.columns[[22, 21, 16]],
    axis=1,
)

notMerged_Budget = budgetExcel_Ids[
    ~budgetExcel_Ids[budgetExcel_Ids.columns[15]].isin(planExcel[planExcel.columns[7]])
]
notMerged_Budget = pd.concat([notMerged_Budget, budgetExcel_no_Ids])
notMerged_Plan = planExcel[
    ~planExcel[planExcel.columns[7]].isin(budgetExcel_Ids[budgetExcel_Ids.columns[15]])
]





Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                    Unnamed: 0_level_1 Unnamed: 1_level_1   
19   Hakaniementori + (Länsipää Miina Sillanpään katu)               K5.1   
20             Siltasaarenkatu (Hakaniemenr - Hämeent)               K5.1   
21          Signe Branderin terassi (itä+länsi) + kuja               K5.1   
22                                  Näkinkulku (pinta)               K5.1   
23     John Stenbergin ranta (esirakentamisesta paalu)               K5.1   
..                                                 ...                ...   
619                                         Mätäspolku                 K4   
620      Maatullinkj (Henrik Forsiuksen t - Kämnerink)                 K4   
624                          Muut peruskorjaushankkeet                 K4   
625                            Pienet peruskorjaustyöt                 K4   
626                       Katupuiden korvausistutukset                 K4   

    asuntotuotan. ennuste   2023   2024   2025   2026   2027   2028   2029  \
       Kyllä / Ei  1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 €   
19          False  3000.0   None   None   None   None   None   None   None   
20           True   500.0   None   None   None   None   None   None   None   
21           True  3500.0   None   None   None   None   None   None   None   
22           True   300.0   None   None   None   None   None   None   None   
23           True  1500.0   None   None   None   None   None   None    0.0   
..            ...     ...    ...    ...    ...    ...    ...    ...    ...   
619         False   400.0   None   None   None   None   None   None   None   
620         False   900.0   None   None   None   None   None   None   None   
624         False  2000.0   None    0.0   None   None   None   None   None   
625         False  1000.0   None   None   None   None   None   None   None   
626         False   200.0   None   None   None   None   None   None   None   

      2030   2031   2032                                                   \
    1000 € 1000 € 1000 €                                         1000 €.1   
19    None   None   None  Suunnittelun aloitus vasta 25 ja toteutus 26+27   
20    None   None   None                                             None   
21    None   None   None                                             None   
22    None   None   None                                             None   
23    None   None   None                                        Tarve 2,0   
..     ...    ...    ...                                              ...   
619   None   None   None                                             None   
620   None   None   None                                             None   
624   None   None   None                                             None   
625   None   None   None                                             None   
626   None   None   None                                             None   

       HKR / PW  
    HANKENUMERO  
19          191  
20          839  
21         3174  
22         3132  
23         1976  
..          ...  
619        None  
620        None  
624        None  
625        None  
626        None  

[361 rows x 16 columns]

In [76]:
notMerged_Plan
# notMerged_Budget
# merged_Plan_Budget

,Unnamed: 0_level_0,LASKE-SAP-,Verkko-,Alue-,Kohteen,Hankkeen,Huom.,HKR / PW
,HANKE,projekti,numero,vastaava,projektipäällikkö,TS2023,Unnamed: 26_level_1,HANKENUMERO
11,Puutarhakuja,2814I00019,6705806,None,Kimari,140,suunnitelmat 3/23,None
12,Mannerheimintie 3-5,None,None,None,Kimari,30,pienet järjestelyt hotellin edustalla,None
14,"Kaartinkortteli, Fabianinkatu 2",None,None,None,NN,270,None,None
16,Keskustan THK,None,None,Kaalikoski,None,50,kaavasta valitettu,None
17,Keskustan THK liittyvät kadut,None,None,None,Kaalikoski,50,tarvitaan maisemaark. näkemystä mukaan aukion ...,None
...,...,...,...,...,...,...,...,...
724,Vanhankaupunginkoski ja Kuninkaansaari,2814I03214,6704221,None,Luomanen,None,None,None
726,Pyörökivenpuiston tulvasuojelu,2814I03210,6703192,None,NN,50,None,None
727,"Idän rantareitti, perusparannus v. Reginankuja...",2814I03216,6705620,None,Knuuti,540,None,None


In [74]:
notMerged_Budget_data = [
    {
        "name": name,
        "sapProject": sapProject,
        "sapNetwork": sapNetwork,
        "costForecast": costForecast,
        "description":"To be filled",
        "budgetProposalCurrentYearPlus1": bpCPlus1,
        "budgetProposalCurrentYearPlus2": bpCPlus2,
        "preliminaryCurrentYearPlus3": pcyPlus3,
        "preliminaryCurrentYearPlus4": pcyPlus4,
        "preliminaryCurrentYearPlus5": pcyPlus5,
        "preliminaryCurrentYearPlus6": pcyPlus6,
        "preliminaryCurrentYearPlus7": pcyPlus7,
        "preliminaryCurrentYearPlus8": pcyPlus8,
        "preliminaryCurrentYearPlus9": pcyPlus9,
        "preliminaryCurrentYearPlus10": pcyPlus10,
        "note_content": note_content,
        "hkrId": hkrId,
    }
    for name, sapProject, sapNetwork, costForecast, bpCPlus1, bpCPlus2, pcyPlus3, pcyPlus4, pcyPlus5, pcyPlus6, pcyPlus7, pcyPlus8, pcyPlus9, pcyPlus10, note_content, hkrId in zip(
        notMerged_Budget.iloc[:, 0],
        notMerged_Budget.iloc[:, 1],
        notMerged_Budget.iloc[:, 2],
        notMerged_Budget.iloc[:, 3],
        notMerged_Budget.iloc[:, 4],
        notMerged_Budget.iloc[:, 5],
        notMerged_Budget.iloc[:, 6],
        notMerged_Budget.iloc[:, 7],
        notMerged_Budget.iloc[:, 8],
        notMerged_Budget.iloc[:, 9],
        notMerged_Budget.iloc[:, 10],
        notMerged_Budget.iloc[:, 11],
        notMerged_Budget.iloc[:, 12],
        notMerged_Budget.iloc[:, 13],
        notMerged_Budget.iloc[:, 14],
        notMerged_Budget.iloc[:, 15],
    )
]



In [77]:
notMerged_Plan_data = [
    {
        "name": name,
        "category": category,
        "effectHousing": effectHousing,
        "personConstruction": personConstruction,
        "personPlanning": personPlanning,
        "costForecast": costForecast,
        "note_content": note_content,
        "description":"To be filled",
        "hkrId": hkrId,
    }
    for name, category, effectHousing, personConstruction, personPlanning, costForecast, note_content, hkrId in zip(
        notMerged_Plan.iloc[:, 0],
        notMerged_Plan.iloc[:, 1],
        notMerged_Plan.iloc[:, 2],
        notMerged_Plan.iloc[:, 3],
        notMerged_Plan.iloc[:, 4],
        notMerged_Plan.iloc[:, 5],
        notMerged_Plan.iloc[:, 6],
        notMerged_Plan.iloc[:, 7],
       
    )
]

In [78]:
notMerged_Plan_data

[{'name': 'Puutarhakuja',
  'category': '2814I00019',
  'effectHousing': 6705806,
  'personConstruction': None,
  'personPlanning': 'Kimari',
  'costForecast': 140,
  'note_content': 'suunnitelmat 3/23',
  'description': 'To be filled',
  'hkrId': None},
 {'name': 'Mannerheimintie 3-5',
  'category': None,
  'effectHousing': None,
  'personConstruction': None,
  'personPlanning': 'Kimari',
  'costForecast': 30,
  'note_content': 'pienet järjestelyt hotellin edustalla',
  'description': 'To be filled',
  'hkrId': None},
 {'name': 'Kaartinkortteli, Fabianinkatu 2',
  'category': None,
  'effectHousing': None,
  'personConstruction': None,
  'personPlanning': 'NN',
  'costForecast': 270,
  'note_content': None,
  'description': 'To be filled',
  'hkrId': None},
 {'name': 'Keskustan THK',
  'category': None,
  'effectHousing': None,
  'personConstruction': 'Kaalikoski',
  'personPlanning': None,
  'costForecast': 50,
  'note_content': 'kaavasta valitettu',
  'description': 'To be filled',
